查看npy文件内容

In [ ]:
import argparse
import os
import shutil

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import traceback
import sys

from data_cnn60 import AverageMeter, NTUDataLoaders
from s_model import (MLP, Decoder, Discriminator, Encoder, KL_divergence,
                   permute_dims, reparameterize)

In [ ]:
def import_class(import_str):
    mod_str, _sep, class_str = import_str.rpartition('.')
    __import__(mod_str)
    try:
        return getattr(sys.modules[mod_str], class_str)
    except AttributeError:
        raise ImportError('Class %s cannot be found (%s)' % (class_str, traceback.format_exception(*sys.exc_info())))


finegrain_model = import_class("model.shiftgcn_match_ntu.ModelMatch")

In [ ]:
# load the semantic attributes
# attribute_features_dict = torch.load('/DATA3/cy/STAR/data/text_feature/ntu_spatial_temporal_attribute_feature_dict_gpt35.tar')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
action_descriptions = torch.load('text_feature/ntu_semantic_part_feature_dict_gpt35_6part.tar')

# load part language description
part_language = []
for i, part_name in enumerate(["head", "hand", "arm", "hip", "leg", "foot"]):
    part_language.append(action_descriptions[i+1].unsqueeze(1))
part_language1 = torch.cat(part_language, dim=1).cuda(device)